In [ ]:
## For pedestal ADC distributions

In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import os

# The ROOT file
file = uproot.open("/Users/jasminechhikara/Desktop/run_20241018_173524/pedestal_run0.root")   # change the name of the folder here 
# Accessing the TTree 
tree = file["unpacker_data/hgcroc"]

# Accessing 'channel' and 'ADC' branches
local_channels = tree["channel"].array()
adc_values = tree["adc"].array()  
chip = tree["chip"].array()
half = tree["half"].array()

# Global channel ID
channels = (chip*2*39) + (half*39) + local_channels

# Directory to save the plots
output_dir = "Pedestal_out"
os.makedirs(output_dir, exist_ok=True)

# Unique channels
unique_channels = np.unique(channels)

# Define list to store channel numbers and corresponding mean ADC values
channel_numbers = []
mean_adc_values = []

# ADC distribution for each channel and the mean
for channel in unique_channels:
    # ADC values corresponding to the channel
    adc_for_channel = adc_values[channels == channel].to_numpy()  # Convert to NumPy array

    # Store the channel number and mean ADC value
    channel_numbers.append(channel)
    mean_adc_values.append(mean_adc)
    
    # Bin width
    bin_edges = np.arange(adc_for_channel.min(), adc_for_channel.max() + 2)  

    # Histogram with 1 ADC bin width
    plt.figure()
    plt.hist(adc_for_channel, bins=bin_edges, alpha=0.75, label=f"Channel {channel}")
    plt.xlabel("ADC", fontsize=14)
    plt.ylabel("Number of entries", fontsize=14)
    plt.title(f"ADC Distribution for Channel {channel}", fontsize=14)
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Save the plot 
    plt.savefig(os.path.join(output_dir, f"ADC_dist_{channel}.png"))
    plt.close()

# The mean ADC values against the channel numbers
plt.figure()
plt.scatter(channel_numbers, mean_adc_values, marker='o', linestyle='-', color='b')
plt.xlabel("Channel Number",fontsize=14)
plt.ylabel("Mean ADC ",fontsize=14)
plt.title("Mean ADC Value vs Channel",fontsize=14)
plt.grid(True)

# Save Mean ADC values plot
plt.savefig(os.path.join(output_dir, "Mean_ADC_Values.png"))
plt.show()


In [ ]:
## For L1Offset scan : to know the maximum reponse BX

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os

# Directory with the ROOT files
input_dir = "/Users/jasminechhikara/Desktop/run_20241125_230434"  # Replace with the directory containing l1offset_scan*.root files
output_dir = "Mean_ADC_vs_BX_205034"
os.makedirs(output_dir, exist_ok=True)

# Files to process
file_list = [f"l1offset_scan{i}.root" for i in range(20)]

# Define dictionaries to store ADC values per channel
channel_adc_values = {}
channel_chip_half = {}

# Looping through each file 
for file_name in file_list:
    file_path = os.path.join(input_dir, file_name)
    
    # Loading the ROOT file
    file = uproot.open(file_path)
    
    # Accessing the TTree
    tree = file["unpacker_data/hgcroc"]
    
    # # Accessing 'channel' , 'half' , 'chip' , 'ADC' branches
    local_channels = tree["channel"].array()
    adc_values = tree["adc"].array()
    chip = tree["chip"].array()
    half = tree["half"].array()
    
    # Global channel ID
    channels = (chip*2*39) + (half*39) + local_channels
    
    # Updating channel_adc_values and channel_chip_half dictionaries
    for channel in np.unique(channels):
        if channel not in channel_adc_values:
            channel_adc_values[channel] = []
            # Store the chip and half values for the first occurrence of this channel
            chip_value = chip[channels == channel][0]
            half_value = half[channels == channel][0]
            channel_chip_half[channel] = (chip_value, half_value)
        
        # ADC values for the channel
        adc_for_channel = adc_values[channels == channel]
        
        # Mean ADC value for this channel to the dictionary
        channel_adc_values[channel].append(np.mean(adc_for_channel))


# Plot for each channel
for channel, adc_values in channel_adc_values.items():
    plt.figure()
    plt.bar(range(len(adc_values)), adc_values, alpha=0.75, color='blue', align='center')
    plt.xlabel("Bunch crossing (BX) number",fontsize=14)
    plt.ylabel("Mean ADC",fontsize=14)
    
    # Chip and half values
    chip_value, half_value = channel_chip_half[channel]

    plt.title(f"Mean ADC vs. BX for Channel {channel}\nChip: {chip_value}, Half: {half_value}",fontsize=14)
    plt.grid(True)
    plt.xticks(ticks=range(len(adc_values)), labels=range(len(adc_values)))
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.savefig(os.path.join(output_dir, f"Mean_ADC_vs_BX_{channel}.png"))
    plt.show()




In [ ]:
## For phase scan : to know the maximum reponse phase

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os
import time

start = time.time()

# Directory wiht the ROOT files
input_dir = "/Users/jasminechhikara/Desktop/run_20241125_222206/"   ## change the input directory info here
output_dir = "Mean_ADC_vs_phase_222206"
os.makedirs(output_dir, exist_ok=True)

# List of files 
file_list = [f"phase_scan_withBeam{i}.root" for i in range(16)]

# Create dictionaries to store ADC values per channel
channel_adc_values = {}

# Loop through each file 
for file_name in file_list:
    file_path = os.path.join(input_dir, file_name)
    
    # Load the ROOT file
    file = uproot.open(file_path)
    
    # Access the TTree 
    tree = file["unpacker_data/hgcroc"]
    
    # Data from the 'channel' , 'adc' , 'chip' and 'half' branches
    local_channels = tree["channel"].array()
    adc_values = tree["adc"].array()
    chip = tree["chip"].array()
    half = tree["half"].array()
    
    # Global channel ID
    channels = (chip * 2 * 39) + (half * 39) + local_channels
    
    # Update channel_adc_values dictionary
    for channel in np.unique(channels):
        if channel not in channel_adc_values:   # This line checks if the current channel ID is already a key in the dictionary channel_adc_values.
            channel_adc_values[channel] = {"adc": [], "chip": chip[channels == channel][0], "half": half[channels == channel][0]}   # If the channel is not already in the dictionary, it initializes an empty list for that channel. This list is intended to store the mean ADC values for this channel across all phase runs
            
        # Extract ADC values for channel
        adc_for_channel = adc_values[channels == channel]
        # The mean ADC value for this channel to the dictionary
        channel_adc_values[channel]["adc"].append(np.mean(adc_for_channel))

# Plot for each channel
for channel, data in channel_adc_values.items():
    adc_values = data["adc"]
    chip = data["chip"]
    half = data["half"]
    
    plt.figure()
    plt.bar(range(len(adc_values)), adc_values, alpha=0.75, color='blue', align='center')
    plt.xlabel("Phase number")
    plt.ylabel("Mean ADC Value")
    plt.title(f"Mean ADC Value vs. Phase number for Channel {channel} (Chip: {chip}, Half: {half})")
    plt.grid(True)
    plt.xticks(ticks=range(len(adc_values)), labels=range(len(adc_values)))
    plt.savefig(os.path.join(output_dir, f"Mean_ADC_vs_phase_number_Channel_{channel}_Chip_{chip}_Half_{half}.png"))
    plt.close()
end = time.time()
print(start-end)


In [ ]:
## Pedestal subracted ADC values

In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import os

# File paths
pedestal_file = "/Users/jasminechhikara/Desktop/run_20241125_230339/pedestal_run0.root"     # directory with pedestal run file
beam_file = "/Users/jasminechhikara/Desktop/run_20241126_120911/beam_run0.root"  ## directory with beam run file

# Load ROOT files
pedestal_tree = uproot.open(f"{pedestal_file}")["unpacker_data/hgcroc"]
beam_tree = uproot.open(f"{beam_file}")["unpacker_data/hgcroc"]

# Extracting 'half' , 'chip' , 'channel' , 'ADC' branches
pedestal_channels = pedestal_tree["channel"].array()
pedestal_adc = pedestal_tree["adc"].array()
pedestal_chip = pedestal_tree["chip"].array()
pedestal_half = pedestal_tree["half"].array()

beam_channels = beam_tree["channel"].array()
beam_adc = beam_tree["adc"].array()
beam_chip = beam_tree["chip"].array()
beam_half = beam_tree["half"].array()

channel_bad = []

# Calculate global channel IDs
pedestal_global_channels = (pedestal_chip * 2 * 39) + (pedestal_half * 39) + pedestal_channels
beam_global_channels = (beam_chip * 2 * 39) + (beam_half * 39) + beam_channels

# Calculate mean pedestal ADC values for each global channel
pedestal_means = {}
for channel in np.unique(pedestal_global_channels):
    adc_values = pedestal_adc[pedestal_global_channels == channel]
    pedestal_means[channel] = np.mean(adc_values)

# Directory to save the plots
output_dir = "BeamMinusPedestal_old_2V"
os.makedirs(output_dir, exist_ok=True)

# Subtract pedestal ADC and calculate pedestal subtracted mean ADC values
unique_channels = np.unique(beam_global_channels)
adjusted_means = []
channel_numbers = []

for channel in unique_channels:
    beam_values = beam_adc[beam_global_channels == channel].to_numpy()
    pedestal_mean = pedestal_means.get(channel, 0)  # Default to 0 if no pedestal value exists
    beam_values_mean = np.mean(beam_values)
    adjusted_mean = beam_values_mean - pedestal_mean
    adjusted_values = beam_values - pedestal_mean

    # threshold to differentiate between the channels responding to external injection
    if adjusted_mean < 100:
        channel_bad.append(channel)
    # Store results
    adjusted_means.append(adjusted_mean)
    channel_numbers.append(channel)
    
    # Plot pedestal ADC distribution
    bin_edges = np.arange(adjusted_values.min(), adjusted_values.max() + 2)
    plt.figure()
    plt.hist(adjusted_values, bins=bin_edges, alpha=0.75, label=f"Channel {channel}")
    plt.xlabel("Adjusted ADC", fontsize=14)
    plt.ylabel("Number of entries", fontsize=14)
    plt.title(f"Pedestal subtracted ADC Distribution for Channel {channel}", fontsize=14)
    plt.legend()
    plt.savefig(os.path.join(output_dir, f"Adjusted_ADC_{channel}.png"))
    plt.close()

print(channel_bad)
print(len(channel_bad))
# Plot adjusted mean ADC values against channel numbers
plt.figure()
plt.scatter(channel_numbers, adjusted_means, marker='o', color='b')
plt.xlabel("Channel Number", fontsize=14)
plt.ylabel("Adjusted Mean ADC", fontsize=14)
plt.title("Adjusted Mean ADC Value vs Channel", fontsize=14)
plt.grid(True)
plt.savefig(os.path.join(output_dir, "Adjusted_Mean_ADC_Values.png"))
plt.show()
